In [4]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)   

import traceback
import warnings
warnings.filterwarnings('ignore')

In [5]:
from odps import ODPS
o = ODPS(access_id="LTAIzzPZLTszbygr",
         secret_access_key="5ZOzHa5iNMWBouSLnQHWtdfFVjoWcW",
         project="cashbus_etl",
         endpoint='http://service.cn.maxcompute.aliyun.com/api',
         tunnel_endpoint='http://dt.cn-beijing.maxcompute.aliyun.com')

In [6]:
project = o.get_project()
print(project)

<Project cashbus_etl>


In [7]:
with o.execute_sql("select distinct code_created,loanid,l.loanstyle,code,cast(value as double) as value \
                    from dw_cashbus_rule_code c \
                    left join (select distinct id,loanstyle from loan)l on c.loanid = l.id \
                    where code = 'S991_ALL_C' \
                    and code_created < datetrunc(getdate(), 'dd') \
                    and code_created >= dateadd(datetrunc(getdate(), 'dd'),-2,'dd') \
                    and rulename = 'yszx.l0' \
                    and value not in ('-1','-2') ").open_reader(limit_enabled=False) as reader:
    df_S991_ALL_C = reader.to_pandas()

In [8]:
def df_self_replace(df,old_upper,add_pct):
    sort = df.groupby(df).size()
    df_sort = {'value':sort.index,'loan_num':sort.values}
    df_sort = pd.DataFrame(df_sort)

    pct_old = (df_sort.loc[(df_sort['value'] > 0) & (df_sort['value'] < old_upper)]['loan_num'].sum()/df_sort['loan_num'].sum()) 
    # print("pct_old: " + str(pct_old))
    
    pct_add = pct_old + add_pct  ##拒绝域增加%
    # print("pct_add: " + str(pct_add))
    
    j = 0
    pct_new = 0
    for i in df_sort.value.sort_values():
        pct_i = df_sort.loc[(df_sort['value'] > 0) & (df_sort['value'] < i)]['loan_num'].sum()/df_sort['loan_num'].sum()
        if pct_i <= pct_add:
            j = i
            pct_new = pct_i

    return j,pct_old,pct_add,pct_new

In [9]:
# print("绿卡30天2期: ")
# print(df_self_replace(df_S991_ALL_C.loc[df_S991_ALL_C.loanstyle == '绿卡30天2期'].value,901,0.03))
# print(df_self_replace(df_S991_ALL_C.loc[df_S991_ALL_C.loanstyle == '绿卡30天2期'].value,870,0.05))
# print("          ")
print("绿卡3月3期: ")
print(df_self_replace(df_S991_ALL_C.loc[df_S991_ALL_C.loanstyle == '绿卡3月3期'].value,878,-0.10))
print(df_self_replace(df_S991_ALL_C.loc[df_S991_ALL_C.loanstyle == '绿卡3月3期'].value,841,-0.10))
print(df_self_replace(df_S991_ALL_C.loc[df_S991_ALL_C.loanstyle == '绿卡3月3期'].value,788,-0.10))

print(df_self_replace(df_S991_ALL_C.loc[df_S991_ALL_C.loanstyle == '绿卡3月3期'].value,910,-0.05))
# print(df_self_replace(df_S991_ALL_C.loc[df_S991_ALL_C.loanstyle == '绿卡3月3期'].value,885,0.02))
# print("          ")
# print("绿卡分期5月: ")
# print(df_self_replace(df_S991_ALL_C.loc[df_S991_ALL_C.loanstyle == '绿卡分期5月'].value,925,0.118))

绿卡3月3期: 
(842.0, 0.7268885367142103, 0.6268885367142103, 0.6261295076041927)
(802.0, 0.6235716073066978, 0.5235716073066978, 0.5217282508966553)
(741.0, 0.48816971112408597, 0.38816971112408594, 0.38791114076792615)
(892.0, 0.8218366836266577, 0.7718366836266577, 0.7699279895459727)
